## OSRM (Open Source Routing Machine)

**Author**: SADDIK Imad
<br/>
**Date**: 28/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Table service](#toc1_)    
  - [Setup](#toc1_1_)    
  - [Using the service](#toc1_2_)    
    - [General options](#toc1_2_1_)    
    - [Making the request](#toc1_2_2_)    
      - [No source & no destination](#toc1_2_2_1_)    
      - [With source & no destination](#toc1_2_2_2_)    
      - [No source & With destination](#toc1_2_2_3_)    
      - [With source & With destination](#toc1_2_2_4_)    

---

# <a id='toc1_'></a>[Table service](#toc0_)

Use this service if you want to computes the duration of the fastest route between all pairs of coordinates. Read more about it [here](https://project-osrm.org/docs/v5.24.0/api/?language=cURL#table-service).

## <a id='toc1_1_'></a>[Setup](#toc0_)

I selected four points from the map, designating one as the center. Using the `Folium` package, I will visualize these points on the map.

In [ ]:
import folium

from folium import Map
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.latitude, point.longitude],
                      popup='Point').add_to(folium_map)

    return folium_map


point_1 = Point(latitude=33.89264295626195, longitude=-5.500305816538693)
point_2 = Point(latitude=33.899915132942326, longitude=-5.520818749583605)
point_3 = Point(latitude=33.891645357611154, longitude=-5.5397637571355105)
center_point = Point(latitude=33.89565560255, longitude=-5.522530349727877)

folium_map = get_folium_map(center_point, [point_1, point_2, point_3])
folium_map

## <a id='toc1_2_'></a>[Using the service](#toc0_)

### <a id='toc1_2_1_'></a>[General options](#toc0_)

The general options are used in every OSRM service, read more about them [here](https://project-osrm.org/docs/v5.5.1/api/?language=cURL#general-options). Here is a table that summarizes the options with their description.


| Parameter | Description |
|-----------|-------------|
| service | One of the following values: `route`, `nearest`, `table`, `match`, `trip`, `tile` |
| version | Version of the protocol implemented by the service. `v1` for all OSRM 5.x installations |
| profile | Mode of transportation, is determined statically by the Lua profile that is used to prepare the data using `osrm-extract`. Typically `car`, `bike` or `foot` if using one of the supplied profiles. |
| coordinates | String of format `{longitude},{latitude};{longitude},{latitude};{longitude},{latitude} ...]` or `polyline({polyline})`. |
| format | Only `json` is supported at the moment. This parameter is optional and defaults to `json`. |

### <a id='toc1_2_2_'></a>[Making the request](#toc0_)

The table endpoint follows this format: `/table/v1/{profile}/{coordinates}?{sources}=[{elem}...];&destinations=[{elem}...]`.

The endpoint computes the duration of the fastest route between all pairs of supplied coordinates.

#### <a id='toc1_2_2_1_'></a>[No source & no destination](#toc0_)

In [18]:
service = 'table'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

In [19]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}"

response = requests.get(url)
response.status_code

200

The `200` status code signifies that the request was successful. Now, let's fetch the route and display it on the map.

In [20]:
data = response.json()
data.keys()

dict_keys(['code', 'destinations', 'durations', 'sources'])

By default, if `sources` and `destinations` are not specified, the `table` service returns the duration for all possible combinations. For example, with 3 points, this results in a 3x3 matrix, totaling 9 combinations.

In [24]:
print(data["durations"])

[[0, 218.9, 300.5], [254.9, 0, 246.3], [303, 302.1, 0]]


The table represents the duration matrix for traveling between three points. The rows indicate the **source** points, and the columns indicate the **destination** points. The value in each cell is the travel duration between the corresponding source and destination. When the source and destination are the same, the duration is 0.

| Source \ Destination | Point 1 | Point 2 | Point 3 |
|-----------------------|---------|---------|---------|
| **Point 1**          | 0       | 218.9   | 300.5   |
| **Point 2**          | 254.9   | 0       | 246.3   |
| **Point 3**          | 303     | 302.1   | 0       |


#### <a id='toc1_2_2_2_'></a>[With source & no destination](#toc0_)

We can add the `sources` option to the table endpoint to force the engine to use that point as the starting point while the rest as the destinations.

In [25]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?sources=0"

response = requests.get(url)
response.status_code

200

In this example, we forced the first point to be the source. This yielded a 1x3 matrix.

In [26]:
data = response.json()
data["durations"]

[[0, 218.9, 300.5]]

The table represents the travel duration from **Point 1** (source) to each of the three points (destinations). The rows indicate the **source**, and the columns indicate the **destinations**.

| Source \ Destination | Point 1 | Point 2 | Point 3 |
|-----------------------|---------|---------|---------|
| **Point 1**          | 0       | 218.9   | 300.5   |

#### <a id='toc1_2_2_3_'></a>[No source & With destination](#toc0_)

We can also provide the `destinations` option to force the destination instead of the source.

In [28]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?destinations=0"

response = requests.get(url)
response.status_code

200

In [29]:
data = response.json()
data["durations"]

[[0], [254.9], [303]]

The table shows the travel duration to **Point 1** (destination) from each of the three points (sources). The rows represent the **sources**, and the single column represents the **destination** (Point 1).

| Source \ Destination | Point 1 |
|-----------------------|---------|
| **Point 1**          | 0       |
| **Point 2**          | 254.9   |
| **Point 3**          | 303     |

#### <a id='toc1_2_2_4_'></a>[With source & With destination](#toc0_)

Finally, we can use both options together

In [30]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?sources=0&destinations=2"

response = requests.get(url)
response.status_code

200

This time, we get only one duration in the end.

In [31]:
data = response.json()
data["durations"]

[[300.5]]